In [1]:
from datasets import load_dataset
from dotenv import load_dotenv
import os
from huggingface_hub import login

# Load environment variables from .env file
load_dotenv()

# Get the Hugging Face API key
huggingface_api_key = os.getenv("HF_TOKEN")

login(token=huggingface_api_key)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
yaml_content = """
pipeline_config_id: runner_config

data:
    taxi:
        data_format: json
        # train_dir: FIM4Science/easytpp-synthetic-1d
        # valid_dir: FIM4Science/easytpp-synthetic-1d
        # test_dir: FIM4Science/easytpp-synthetic-1d
        train_dir: easytpp/taxi
        valid_dir: easytpp/taxi
        test_dir: easytpp/taxi
        data_specs:
          num_event_types: 10
          pad_token_id: 10
          padding_side: right


NHP_train:
  base_config:
    stage: train
    backend: torch
    dataset_id: taxi
    runner_id: std_tpp
    model_id: NHP # model name
    base_dir: './checkpoints/'
  trainer_config:
    batch_size: 256
    max_epoch: 2
    shuffle: False
    optimizer: adam
    learning_rate: 1.e-3
    valid_freq: 1
    use_tfb: True
    metrics: [ 'acc', 'rmse' ]
    seed: 2019
    gpu: -1
  model_config:
    hidden_size: 32
    loss_integral_num_sample_per_step: 20
    thinning:
      num_seq: 10
      num_sample: 1
      num_exp: 500 # number of i.i.d. Exp(intensity_bound) draws at one time in thinning algorithm
      look_ahead_time: 10
      patience_counter: 5 # the maximum iteration used in adaptive thinning
      over_sample_rate: 5
      num_samples_boundary: 5
      dtime_max: 5
      num_step_gen: 1
"""

# Save the content to a file named config.yaml
with open("easytpp_training_config.yaml", "w") as file:
    file.write(yaml_content)

FileNotFoundError: [Errno 2] No such file or directory: 'data/1DHawkes/config.yaml'

In [ ]:
from easy_tpp.config_factory import Config
from easy_tpp.runner import Runner

config = Config.build_from_yaml_file('easytpp_training_config.yaml', experiment_id='NHP_train')

model_runner = Runner.build_from_config(config)

model_runner.run()

2025-04-08 15:59:30,179 - config.py[pid:1632825;line:34:build_from_yaml_file] - CRITICAL: Load pipeline config class RunnerConfig
2025-04-08 15:59:30,182 - runner_config.py[pid:1632825;line:151:update_config] - CRITICAL: train model NHP using CPU with torch backend
2025-04-08 15:59:30,189 - runner_config.py[pid:1632825;line:35:__init__] - INFO: Save the config to ./checkpoints/1632825_8864611124288_250408-155930/NHP_train_output.yaml
2025-04-08 15:59:30,190 - base_runner.py[pid:1632825;line:176:save_log] - INFO: Save the log to ./checkpoints/1632825_8864611124288_250408-155930/log


ValueError: Unsupported source directory format for JSON.

In [ ]:
eval_yaml ="""
pipeline_config_id: runner_config

data:
  1DHawkes:
    data_format: json
    train_dir:  data/1DHawkes/train.json
    valid_dir:  data/1DHawkes/val.json
    test_dir:   data/1DHawkes/test.json

NHP_eval:
  base_config:
    stage: eval
    backend: torch
    dataset_id: taxi
    runner_id: std_tpp
    base_dir: './checkpoints/'
    model_id: NHP
  trainer_config:
    batch_size: 256
    max_epoch: 1
    metrics: [ 'acc', 'rmse' ]
  model_config:
    hidden_size: 32
    use_ln: False
    seed: 2019
    gpu: -1
    pretrained_model_dir: ./checkpoints/60211_133800770625536_250406-054218/models/saved_model
    thinning:
      num_seq: 10
      num_sample: 1
      num_exp: 500 # number of i.i.d. Exp(intensity_bound) draws at one time in thinning algorithm
      look_ahead_time: 10
      patience_counter: 5 # the maximum iteration used in adaptive thinning
      over_sample_rate: 5
      num_samples_boundary: 5
      dtime_max: 5
"""

# Save the content to a file named config.yaml
with open("eval_config.yaml", "w") as file:
    file.write(eval_yaml)